# Generation Data — ENTSO-E Examples

Querying actual generation per type, renewable output, and installed capacity.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from entsoe import Client
from entsoe._mappings import PSR_TYPES

client = Client()

## 1. Generation Mix — France (1 week)

In [ ]:
start = "2024-06-01"
end = "2024-06-08"

df_gen = client.generation.actual(start, end, country="FR")
df_gen["fuel"] = df_gen["psr_type"].map(PSR_TYPES)
df_gen.head()

In [ ]:
fig = px.area(
    df_gen, x="timestamp", y="value", color="fuel",
    title="Generation Mix — France",
    labels={"value": "Generation (MW)", "timestamp": ""},
)
fig.update_layout(legend_title_text="Fuel Type")
fig.show()

## 2. Generation Share (Pie Chart)

In [ ]:
totals = df_gen.groupby("fuel")["value"].sum().reset_index()
totals = totals[totals["value"] > 0]

fig = px.pie(
    totals, names="fuel", values="value",
    title="Generation Share — France, 1-8 Jun 2024",
)
fig.show()

## 3. Solar vs Wind — Daily Patterns

In [ ]:
start = "2024-06-03"
end = "2024-06-06"

df_solar = client.generation.actual(start, end, country="FR", psr_type="B16")
df_solar["fuel"] = "Solar"

df_wind = client.generation.actual(start, end, country="FR", psr_type="B19")
df_wind["fuel"] = "Wind Onshore"

df_ren = pd.concat([df_solar, df_wind], ignore_index=True)

fig = px.line(
    df_ren, x="timestamp", y="value", color="fuel",
    title="Solar vs Wind Onshore — France",
    labels={"value": "Generation (MW)", "timestamp": ""},
)
fig.show()

## 4. Wind & Solar Forecast vs Actual

In [ ]:
df_forecast = client.generation.forecast(start, end, country="FR")
df_forecast["fuel"] = df_forecast["psr_type"].map(PSR_TYPES)

# Compare solar
df_solar_fc = df_forecast[df_forecast["psr_type"] == "B16"]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_solar["timestamp"], y=df_solar["value"],
    name="Actual", line=dict(color="#FFA15A"),
))
fig.add_trace(go.Scatter(
    x=df_solar_fc["timestamp"], y=df_solar_fc["value"],
    name="Forecast", line=dict(color="#FFA15A", dash="dash"),
))
fig.update_layout(
    title="Solar Generation — Forecast vs Actual (France)",
    yaxis_title="Generation (MW)",
    xaxis_title="",
)
fig.show()

## 5. Installed Capacity by Type

In [ ]:
start = "2024-01-01"
end = "2024-12-31"

df_cap = client.generation.installed_capacity(start, end, country="FR")
df_cap["fuel"] = df_cap["psr_type"].map(PSR_TYPES)

latest = df_cap.sort_values("timestamp").groupby("fuel").last().reset_index()
latest = latest[latest["value"] > 0].sort_values("value", ascending=True)

fig = px.bar(
    latest, x="value", y="fuel", orientation="h",
    title="Installed Generation Capacity — France, 2024",
    labels={"value": "Capacity (MW)", "fuel": ""},
)
fig.show()